In [9]:
import pandas as pd

import cornac
from cornac.eval_methods import RatioSplit
from cornac.models import BPR

In [10]:
df = pd.read_csv('../Data/Reviews-clean.csv')[['author_id', 'app_id', 'voted_up']]
df['voted_up'] = df['voted_up'].astype(int)
df.head()

,author_id,app_id,voted_up
0,76561197960432447,10,1
1,76561198071230926,10,1
2,76561198206216352,10,1
3,76561198110801124,10,1
4,76561199813732773,10,1


In [11]:
dataset = cornac.data.Dataset.from_uir(df.to_numpy())

# MF

In [12]:
rs = RatioSplit(df.to_numpy(), test_size=0.1, val_size=0.1)
metrics = [
    cornac.metrics.RMSE(),
    cornac.metrics.MAE(),
    cornac.metrics.MSE(),
    cornac.metrics.AUC(),
    cornac.metrics.FMeasure(10),
    cornac.metrics.HitRatio(10),
    cornac.metrics.MAP(),
    cornac.metrics.MRR(),
    cornac.metrics.NCRR(10),
    cornac.metrics.NDCG(10),
    cornac.metrics.Precision(10),
    cornac.metrics.Recall(10)
]

In [ ]:
cornac.Experiment(eval_method=rs, models=[BPR(name='Unimodal BPR')], metrics=metrics, save_dir='Models').run()


VALIDATION:
...
        | NDCG@10 | Precision@10 | Recall@10 | Time (s)
------- + ------- + ------------ + --------- + --------
BiVAECF |  0.0106 |       0.0027 |    0.0239 |  36.2151

TEST:
...
        | NDCG@10 | Precision@10 | Recall@10 |  Train (s) | Test (s)
------- + ------- + ------------ + --------- + ---------- + --------
BiVAECF |  0.0107 |       0.0028 |    0.0249 | 31212.8873 |  34.6108



In [ ]:
from cornac.hyperopt import Discrete, Continuous
from cornac.hyperopt import RandomSearch

rs_bpr = RandomSearch(
    model=BPR(name='Unimodal BPR'),
    space=[
        Discrete(name="k", values=[50, 100, 200]),
        Discrete(name="max_iter", values=[100, 500, 1000]),
        Continuous(name="learning_rate", low=0.001, high=0.1),
        Continuous(name="lambda_reg", low=0.001, high=0.1),
        Discrete(name="use_bias", values=[True, False]),
    ],
    metric=cornac.metrics.Recall(10),
    eval_method=rs,
    n_trails=60,
)

In [13]:
# Define the experiment
cornac.Experiment(
    eval_method=rs,
    models=[rs_bpr],
    metrics=metrics,
    user_based=False,
    save_dir='Models'
).run()

# Obtain the best params
# print(gs_bpr.best_params)
print(rs_bpr.best_params)


VALIDATION:
...
                 | NDCG@10 | Precision@10 | Recall@10 | Time (s)
---------------- + ------- + ------------ + --------- + --------
RandomSearch_BPR |  0.0522 |       0.0099 |    0.0793 |  78.4569

TEST:
...
                 | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
---------------- + ------- + ------------ + --------- + --------- + --------
RandomSearch_BPR |  0.0542 |       0.0102 |    0.0815 | 6728.9108 |  80.2510

{'k': 200, 'lambda_reg': 0.01096994302791341, 'learning_rate': 0.09152844192182917, 'max_iter': 1000, 'use_bias': False}


In [14]:
uni_bpr_tuned = BPR(name='Unimodal BPR Tuned', **{'k': 200, 'lambda_reg': 0.01096994302791341, 'learning_rate': 0.09152844192182917, 'max_iter': 1000, 'use_bias': False})

cornac.Experiment(
    eval_method=rs,
    models=[uni_bpr_tuned],
    metrics=metrics,
    user_based=False,
    save_dir='Models'
).run()


VALIDATION:
...
                   |    MAE |    MSE |   RMSE |    AUC |  F1@10 | HitRatio@10 |    MAP |    MRR | NCRR@10 | NDCG@10 | Precision@10 | Recall@10 | Time (s)
------------------ + ------ + ------ + ------ + ------ + ------ + ----------- + ------ + ------ + ------- + ------- + ------------ + --------- + --------
Unimodal BPR Tuned | 0.3416 | 0.2944 | 0.5426 | 0.8923 | 0.0172 |      0.0924 | 0.0490 | 0.0542 |  0.0450 |  0.0535 |       0.0100 |    0.0802 | 384.5426

TEST:
...
                   |    MAE |    MSE |   RMSE |    AUC |  F1@10 | HitRatio@10 |    MAP |    MRR | NCRR@10 | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
------------------ + ------ + ------ + ------ + ------ + ------ + ----------- + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
Unimodal BPR Tuned | 0.3448 | 0.2974 | 0.5454 | 0.8887 | 0.0177 |      0.0943 | 0.0492 | 0.0543 |  0.0452 |  0.0542 |       0.0103 |    0.0824 |  162.2159 | 399.4416



In [16]:
uni_bpr_tuned.get_item_vectors().shape[0]

63953

In [17]:
uni_bpr_tuned.get_user_vectors().shape[0]

691842